# Fractal Systems Exploration and Analysis

This notebook provides an interactive exploration of fractal dynamical systems used in the Koopman operator learning project. You can visualize different fractal attractors, analyze their properties, and generate trajectory data for neural network training.

## Contents
1. [Setup and Imports](#setup)
2. [Sierpinski Gasket Exploration](#sierpinski)
3. [Barnsley Fern Exploration](#barnsley)
4. [Julia Set Exploration](#julia)
5. [Trajectory Analysis](#trajectory)
6. [Data Generation for Training](#data-generation)

## Setup and Imports {#setup}

In [ ]:
import sys
import os
from pathlib import Path

# Add src to path for imports
sys.path.append(str(Path().absolute().parent.parent / 'src'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML

# Import project modules
from data.generators.fractal_generator import FractalGenerator
from data.generators.ifs_generator import IFSGenerator
from data.generators.julia_generator import JuliaGenerator
from visualization.fractals.fractal_visualizer import FractalVisualizer
from data.datasets.trajectory_dataset import TrajectoryDataset

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Setup complete! Ready to explore fractals.")

## Sierpinski Gasket Exploration {#sierpinski}

The Sierpinski gasket is generated using an Iterated Function System (IFS) with three contractive transformations.

In [ ]:
# Initialize generators and visualizer
fractal_gen = FractalGenerator()
ifs_gen = IFSGenerator()
visualizer = FractalVisualizer()

def explore_sierpinski(n_points=10000, show_trajectory=False, color_by_iteration=False):
    """Interactive Sierpinski gasket exploration."""
    
    # Generate Sierpinski trajectory
    states, next_states = fractal_gen.generate_sierpinski_trajectories(n_points)
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot attractor
    if color_by_iteration:
        colors = np.arange(len(states))
        scatter = axes[0].scatter(states[:, 0], states[:, 1], c=colors, 
                                s=0.5, alpha=0.7, cmap='viridis')
        plt.colorbar(scatter, ax=axes[0], label='Iteration')
    else:
        axes[0].scatter(states[:, 0], states[:, 1], s=0.5, alpha=0.7, color='blue')
    
    axes[0].set_title(f'Sierpinski Gasket ({n_points:,} points)')
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_aspect('equal')
    
    # Show trajectory if requested
    if show_trajectory and n_points <= 1000:
        trajectory_points = min(100, n_points)
        axes[0].plot(states[:trajectory_points, 0], states[:trajectory_points, 1], 
                    'r-', alpha=0.5, linewidth=0.5, label='Trajectory')
        axes[0].legend()
    
    # Plot phase space (state vs next state)
    sample_size = min(5000, len(states))
    idx = np.random.choice(len(states), sample_size, replace=False)
    
    axes[1].scatter(states[idx, 0], next_states[idx, 0], s=1, alpha=0.6, label='x-component')
    axes[1].scatter(states[idx, 1], next_states[idx, 1], s=1, alpha=0.6, label='y-component')
    axes[1].plot([-1, 1], [-1, 1], 'k--', alpha=0.5, label='Identity')
    
    axes[1].set_title('Phase Space: State vs Next State')
    axes[1].set_xlabel('Current State')
    axes[1].set_ylabel('Next State')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Generated {len(states):,} trajectory points")
    print(f"State space bounds: x ∈ [{states[:, 0].min():.3f}, {states[:, 0].max():.3f}], "
          f"y ∈ [{states[:, 1].min():.3f}, {states[:, 1].max():.3f}]")
    print(f"Mean state: ({states.mean(axis=0)[0]:.3f}, {states.mean(axis=0)[1]:.3f})")
    print(f"State std: ({states.std(axis=0)[0]:.3f}, {states.std(axis=0)[1]:.3f})")

# Create interactive widget
interact(explore_sierpinski,
         n_points=widgets.IntSlider(min=1000, max=50000, step=1000, value=10000),
         show_trajectory=widgets.Checkbox(value=False, description='Show trajectory'),
         color_by_iteration=widgets.Checkbox(value=False, description='Color by iteration'));

## Barnsley Fern Exploration {#barnsley}

The Barnsley fern uses a probabilistic IFS with four transformations to create a fern-like fractal.

In [ ]:
def explore_barnsley(n_points=10000, show_probabilities=False, color_by_transform=False):
    """Interactive Barnsley fern exploration."""
    
    # Generate Barnsley fern trajectory
    states, next_states = fractal_gen.generate_barnsley_trajectories(n_points)
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 8))
    
    # Plot attractor
    if color_by_transform:
        # Color points by which transformation was used (approximation)
        # This is a simplified coloring scheme
        colors = np.zeros(len(states))
        colors[states[:, 1] < 0.16] = 0  # Stem
        colors[(states[:, 1] >= 0.16) & (states[:, 0] < 0)] = 1  # Left leaflets
        colors[(states[:, 1] >= 0.16) & (states[:, 0] >= 0)] = 2  # Right leaflets
        
        scatter = axes[0].scatter(states[:, 0], states[:, 1], c=colors, 
                                s=0.5, alpha=0.7, cmap='Set1')
        plt.colorbar(scatter, ax=axes[0], label='Transform Type')
    else:
        axes[0].scatter(states[:, 0], states[:, 1], s=0.5, alpha=0.7, color='green')
    
    axes[0].set_title(f'Barnsley Fern ({n_points:,} points)')
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_aspect('equal')
    
    # Plot distribution analysis
    axes[1].hist2d(states[:, 0], states[:, 1], bins=50, cmap='Greens', alpha=0.8)
    axes[1].set_title('Point Density Distribution')
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    
    plt.tight_layout()
    plt.show()
    
    if show_probabilities:
        # Show transformation probabilities
        print("Barnsley Fern Transformation Probabilities:")
        print("  f1 (stem): 1%")
        print("  f2 (leaflet): 85%")
        print("  f3 (left leaflet): 7%")
        print("  f4 (right leaflet): 7%")
    
    # Print statistics
    print(f"\nGenerated {len(states):,} trajectory points")
    print(f"State space bounds: x ∈ [{states[:, 0].min():.3f}, {states[:, 0].max():.3f}], "
          f"y ∈ [{states[:, 1].min():.3f}, {states[:, 1].max():.3f}]")
    print(f"Aspect ratio: {(states[:, 1].max() - states[:, 1].min()) / (states[:, 0].max() - states[:, 0].min()):.2f}")

# Create interactive widget
interact(explore_barnsley,
         n_points=widgets.IntSlider(min=1000, max=50000, step=1000, value=15000),
         show_probabilities=widgets.Checkbox(value=False, description='Show probabilities'),
         color_by_transform=widgets.Checkbox(value=False, description='Color by transform'));

## Julia Set Exploration {#julia}

Julia sets are generated by iterating the complex function z_{n+1} = z_n^2 + c for different values of the complex parameter c.

In [ ]:
def explore_julia(c_real=-0.7269, c_imag=0.1889, n_points=10000, 
                 max_iterations=100, escape_radius=2.0):
    """Interactive Julia set exploration."""
    
    # Generate Julia set trajectory
    states, next_states = fractal_gen.generate_julia_trajectories(
        n_points=n_points, c_real=c_real, c_imag=c_imag
    )
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot trajectory in complex plane
    axes[0, 0].scatter(states[:, 0], states[:, 1], s=0.5, alpha=0.7, color='purple')
    axes[0, 0].set_title(f'Julia Set Trajectory\nc = {c_real:.4f} + {c_imag:.4f}i')
    axes[0, 0].set_xlabel('Real')
    axes[0, 0].set_ylabel('Imaginary')
    axes[0, 0].set_aspect('equal')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot trajectory evolution (first 1000 points)
    n_traj = min(1000, len(states))
    colors = np.arange(n_traj)
    scatter = axes[0, 1].scatter(states[:n_traj, 0], states[:n_traj, 1], 
                               c=colors, s=2, alpha=0.8, cmap='plasma')
    axes[0, 1].plot(states[:n_traj, 0], states[:n_traj, 1], 'k-', alpha=0.3, linewidth=0.5)
    plt.colorbar(scatter, ax=axes[0, 1], label='Iteration')
    axes[0, 1].set_title('Trajectory Evolution (First 1000 points)')
    axes[0, 1].set_xlabel('Real')
    axes[0, 1].set_ylabel('Imaginary')
    axes[0, 1].set_aspect('equal')
    
    # Plot magnitude evolution
    magnitudes = np.sqrt(states[:, 0]**2 + states[:, 1]**2)
    axes[1, 0].plot(magnitudes[:min(1000, len(magnitudes))], alpha=0.8)
    axes[1, 0].axhline(y=escape_radius, color='r', linestyle='--', 
                      label=f'Escape radius ({escape_radius})')
    axes[1, 0].set_title('Magnitude Evolution')
    axes[1, 0].set_xlabel('Iteration')
    axes[1, 0].set_ylabel('|z|')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot phase space
    sample_size = min(5000, len(states))
    idx = np.random.choice(len(states), sample_size, replace=False)
    
    axes[1, 1].scatter(states[idx, 0], next_states[idx, 0], s=1, alpha=0.6, label='Real')
    axes[1, 1].scatter(states[idx, 1], next_states[idx, 1], s=1, alpha=0.6, label='Imaginary')
    axes[1, 1].plot([-2, 2], [-2, 2], 'k--', alpha=0.5, label='Identity')
    
    axes[1, 1].set_title('Phase Space: State vs Next State')
    axes[1, 1].set_xlabel('Current State')
    axes[1, 1].set_ylabel('Next State')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Generated {len(states):,} trajectory points")
    print(f"Complex parameter c = {c_real:.4f} + {c_imag:.4f}i")
    print(f"Magnitude range: [{magnitudes.min():.3f}, {magnitudes.max():.3f}]")
    print(f"Mean magnitude: {magnitudes.mean():.3f} ± {magnitudes.std():.3f}")
    
    # Check if trajectory is bounded
    if magnitudes.max() < escape_radius:
        print("✓ Trajectory appears bounded (Julia set member)")
    else:
        print("⚠ Trajectory may be escaping to infinity")

# Create interactive widget with interesting Julia set parameters
interact(explore_julia,
         c_real=widgets.FloatSlider(min=-1.5, max=0.5, step=0.01, value=-0.7269),
         c_imag=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.1889),
         n_points=widgets.IntSlider(min=1000, max=20000, step=1000, value=10000),
         max_iterations=widgets.IntSlider(min=50, max=200, step=10, value=100),
         escape_radius=widgets.FloatSlider(min=1.5, max=5.0, step=0.1, value=2.0));

### Interesting Julia Set Parameters

Try these interesting values for the complex parameter c:

- **Dragon**: c = -0.7269 + 0.1889i (default)
- **Rabbit**: c = -0.123 + 0.745i
- **Airplane**: c = -0.7 + 0.27015i
- **Spiral**: c = -0.75 + 0.11i
- **Dendrite**: c = -0.235 + 0.827i

## Trajectory Analysis {#trajectory}

Analyze the dynamical properties of fractal trajectories.

In [ ]:
def analyze_trajectory_properties(system_type='sierpinski', n_points=10000):
    """Analyze dynamical properties of fractal trajectories."""
    
    # Generate trajectory data
    if system_type == 'sierpinski':
        states, next_states = fractal_gen.generate_sierpinski_trajectories(n_points)
    elif system_type == 'barnsley':
        states, next_states = fractal_gen.generate_barnsley_trajectories(n_points)
    elif system_type == 'julia':
        states, next_states = fractal_gen.generate_julia_trajectories(n_points)
    else:
        raise ValueError(f"Unknown system type: {system_type}")
    
    # Create analysis plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # 1. State distribution
    axes[0, 0].hist(states[:, 0], bins=50, alpha=0.7, label='x-component', density=True)
    axes[0, 0].hist(states[:, 1], bins=50, alpha=0.7, label='y-component', density=True)
    axes[0, 0].set_title('State Component Distributions')
    axes[0, 0].set_xlabel('Value')
    axes[0, 0].set_ylabel('Density')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Step size distribution
    step_sizes = np.linalg.norm(next_states - states, axis=1)
    axes[0, 1].hist(step_sizes, bins=50, alpha=0.7, density=True)
    axes[0, 1].set_title('Step Size Distribution')
    axes[0, 1].set_xlabel('Step Size')
    axes[0, 1].set_ylabel('Density')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Autocorrelation
    max_lag = min(100, len(states) // 10)
    lags = np.arange(max_lag)
    autocorr_x = [np.corrcoef(states[:-lag, 0], states[lag:, 0])[0, 1] if lag > 0 
                  else 1.0 for lag in lags]
    autocorr_y = [np.corrcoef(states[:-lag, 1], states[lag:, 1])[0, 1] if lag > 0 
                  else 1.0 for lag in lags]
    
    axes[0, 2].plot(lags, autocorr_x, label='x-component', marker='o', markersize=3)
    axes[0, 2].plot(lags, autocorr_y, label='y-component', marker='s', markersize=3)
    axes[0, 2].axhline(y=0, color='k', linestyle='--', alpha=0.5)
    axes[0, 2].set_title('Autocorrelation Function')
    axes[0, 2].set_xlabel('Lag')
    axes[0, 2].set_ylabel('Correlation')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Return map (Poincaré plot)
    sample_size = min(5000, len(states))
    idx = np.random.choice(len(states), sample_size, replace=False)
    
    axes[1, 0].scatter(states[idx, 0], next_states[idx, 0], s=1, alpha=0.6, label='x')
    axes[1, 0].scatter(states[idx, 1], next_states[idx, 1], s=1, alpha=0.6, label='y')
    axes[1, 0].plot([states.min(), states.max()], [states.min(), states.max()], 
                   'k--', alpha=0.5, label='Identity')
    axes[1, 0].set_title('Return Map (Poincaré Plot)')
    axes[1, 0].set_xlabel('x(t)')
    axes[1, 0].set_ylabel('x(t+1)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5. Trajectory segments
    n_segments = 5
    segment_length = min(200, len(states) // n_segments)
    colors = plt.cm.viridis(np.linspace(0, 1, n_segments))
    
    for i in range(n_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        axes[1, 1].plot(states[start_idx:end_idx, 0], states[start_idx:end_idx, 1], 
                       color=colors[i], alpha=0.7, linewidth=1, 
                       label=f'Segment {i+1}')
    
    axes[1, 1].set_title('Trajectory Segments')
    axes[1, 1].set_xlabel('x')
    axes[1, 1].set_ylabel('y')
    axes[1, 1].legend()
    axes[1, 1].set_aspect('equal')
    
    # 6. Spectral analysis (power spectrum)
    from scipy import signal
    
    # Compute power spectral density
    freqs_x, psd_x = signal.welch(states[:, 0], nperseg=min(1024, len(states)//4))
    freqs_y, psd_y = signal.welch(states[:, 1], nperseg=min(1024, len(states)//4))
    
    axes[1, 2].loglog(freqs_x[1:], psd_x[1:], label='x-component', alpha=0.8)
    axes[1, 2].loglog(freqs_y[1:], psd_y[1:], label='y-component', alpha=0.8)
    axes[1, 2].set_title('Power Spectral Density')
    axes[1, 2].set_xlabel('Frequency')
    axes[1, 2].set_ylabel('PSD')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print numerical statistics
    print(f"\n{system_type.title()} Trajectory Analysis")
    print("=" * 40)
    print(f"Number of points: {len(states):,}")
    print(f"State space bounds: x ∈ [{states[:, 0].min():.4f}, {states[:, 0].max():.4f}]")
    print(f"                   y ∈ [{states[:, 1].min():.4f}, {states[:, 1].max():.4f}]")
    print(f"Mean step size: {step_sizes.mean():.6f} ± {step_sizes.std():.6f}")
    print(f"Max step size: {step_sizes.max():.6f}")
    print(f"Correlation x-y: {np.corrcoef(states[:, 0], states[:, 1])[0, 1]:.4f}")
    
    # Estimate fractal dimension (box counting approximation)
    def box_count_dimension(points, scales):
        """Approximate box counting dimension."""
        counts = []
        for scale in scales:
            # Discretize points
            discretized = np.floor(points / scale).astype(int)
            # Count unique boxes
            unique_boxes = len(np.unique(discretized, axis=0))
            counts.append(unique_boxes)
        return counts
    
    scales = np.logspace(-3, -1, 10)
    counts = box_count_dimension(states, scales)
    
    # Fit power law to estimate dimension
    log_scales = np.log(1/scales)
    log_counts = np.log(counts)
    
    # Linear fit
    coeffs = np.polyfit(log_scales[2:-2], log_counts[2:-2], 1)
    estimated_dim = coeffs[0]
    
    print(f"Estimated fractal dimension: {estimated_dim:.2f}")

# Create interactive widget
interact(analyze_trajectory_properties,
         system_type=widgets.Dropdown(options=['sierpinski', 'barnsley', 'julia'], 
                                     value='sierpinski'),
         n_points=widgets.IntSlider(min=5000, max=30000, step=2500, value=15000));

## Data Generation for Training {#data-generation}

Generate and save trajectory datasets for neural network training.

In [ ]:
def generate_training_data(systems=['sierpinski', 'barnsley', 'julia'],
                          n_points_per_system=20000,
                          save_data=True,
                          output_dir='../../data'):
    """Generate training datasets for all fractal systems."""
    
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    generated_datasets = {}
    
    for system in systems:
        print(f"\nGenerating {system} dataset...")
        
        # Generate trajectory data
        if system == 'sierpinski':
            states, next_states = fractal_gen.generate_sierpinski_trajectories(n_points_per_system)
        elif system == 'barnsley':
            states, next_states = fractal_gen.generate_barnsley_trajectories(n_points_per_system)
        elif system == 'julia':
            states, next_states = fractal_gen.generate_julia_trajectories(n_points_per_system)
        else:
            print(f"Unknown system: {system}")
            continue
        
        # Create dataset
        dataset = TrajectoryDataset(
            states=states,
            next_states=next_states,
            train_ratio=0.7,
            val_ratio=0.15,
            test_ratio=0.15,
            normalize=True,
            seed=42
        )
        
        generated_datasets[system] = {
            'states': states,
            'next_states': next_states,
            'dataset': dataset
        }
        
        # Save data if requested
        if save_data:
            # Save raw trajectory data
            trajectory_data = np.column_stack([states, next_states])
            trajectory_path = output_path / f'{system}_trajectories.npy'
            np.save(trajectory_path, trajectory_data)
            
            # Save as CSV for external tools
            csv_path = output_path / f'{system}_trajectories.csv'
            np.savetxt(csv_path, trajectory_data, delimiter=',', 
                      header='x,y,next_x,next_y', comments='')
            
            print(f"  Saved to {trajectory_path}")
            print(f"  Saved to {csv_path}")
        
        print(f"  Generated {len(states):,} trajectory points")
        print(f"  Train/Val/Test split: {len(dataset.train_loader.dataset)}/"
              f"{len(dataset.val_loader.dataset)}/{len(dataset.test_loader.dataset)}")
    
    # Create summary visualization
    if generated_datasets:
        fig, axes = plt.subplots(1, len(generated_datasets), 
                               figsize=(5*len(generated_datasets), 5))
        
        if len(generated_datasets) == 1:
            axes = [axes]
        
        for i, (system, data) in enumerate(generated_datasets.items()):
            states = data['states']
            axes[i].scatter(states[:, 0], states[:, 1], s=0.5, alpha=0.7)
            axes[i].set_title(f'{system.title()} ({len(states):,} points)')
            axes[i].set_xlabel('x')
            axes[i].set_ylabel('y')
            axes[i].set_aspect('equal')
        
        plt.tight_layout()
        plt.show()
    
    print(f"\nData generation complete!")
    print(f"Total systems: {len(generated_datasets)}")
    print(f"Total points: {sum(len(data['states']) for data in generated_datasets.values()):,}")
    
    if save_data:
        print(f"All data saved to: {output_path}")
    
    return generated_datasets

# Create interactive data generation widget
systems_widget = widgets.SelectMultiple(
    options=['sierpinski', 'barnsley', 'julia'],
    value=['sierpinski', 'barnsley'],
    description='Systems:'
)

n_points_widget = widgets.IntSlider(
    min=5000, max=50000, step=2500, value=20000,
    description='Points per system:'
)

save_widget = widgets.Checkbox(
    value=True, description='Save data to files'
)

output_dir_widget = widgets.Text(
    value='../../data', description='Output directory:'
)

# Create interactive interface
interactive_plot = interactive(generate_training_data,
                             systems=systems_widget,
                             n_points_per_system=n_points_widget,
                             save_data=save_widget,
                             output_dir=output_dir_widget)

display(interactive_plot)

## Summary

This notebook provides comprehensive tools for exploring fractal dynamical systems:

1. **Interactive Visualization**: Explore different fractal attractors with adjustable parameters
2. **Trajectory Analysis**: Analyze dynamical properties including autocorrelation, return maps, and spectral characteristics
3. **Data Generation**: Create training datasets for neural network experiments

The generated data can be used with the neural network training scripts to learn Koopman operators for these fractal systems.

### Next Steps

1. Use the generated data with the training scripts in `experiments/scripts/`
2. Explore the model comparison notebook for analyzing trained models
3. Check the results demonstration notebook for key findings

### Key Insights

- **Sierpinski Gasket**: Self-similar structure with clear geometric patterns
- **Barnsley Fern**: Probabilistic IFS with natural, organic appearance
- **Julia Sets**: Complex dynamics with rich boundary structures

Each system presents unique challenges for Koopman operator learning, making them excellent test cases for comparing neural network architectures.